<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from HanTa import HanoverTagger as ht

<p>Download der Stopwörter Listen</p>

In [72]:
#nltk.download('stopwords')

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [117]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager_de = df_manager_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [3]:
df_manager_de.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  701784983829143553   554491983 2016-02-22 15:05:53+00:00   
3  701446667170615296  3365669884 2016-02-21 16:41:32+00:00   
4  700597812556386304  4858383167 2016-02-19 08:28:29+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  701784983829143553   de              0            0           0   
3  701446667170615296   de              0            0           1   
4  700597812556386304   de              2            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter Web Client       None                None   
1            0  Twitter Web Client       None                None   
2            0            Facebook       None                None   
3            0  Twitter Web Client       None                None   
4            0  Twitter for iPhone  retweeted  700578193334411264   

                                                text  
0  #Wirecard-Chef Markus Braun pumpt irrwitzige S...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Morgen startet die neue Reihe R²-JazzCube! Vol...  
3  die Y-Achse des  #prusai3 nimmt Formen an :-) ...  
4  RT @aktionaer: #Wirecard: Was führt der Vorsta...

<br>
<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [3]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

conversation_id:  703311366262800384 type:  <class 'numpy.int64'>
author_id:  1392131804 type:  <class 'numpy.int64'>
created_at:  2016-02-26 20:11:11+00:00 type:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
tweet_id:  703311366262800384 type:  <class 'numpy.int64'>
lang:  de type:  <class 'str'>
retweet_count:  0 type:  <class 'numpy.int64'>
reply_count:  0 type:  <class 'numpy.int64'>
like_count:  2 type:  <class 'numpy.int64'>
quote_count:  0 type:  <class 'numpy.int64'>
source:  Twitter Web Client type:  <class 'str'>
tweet_type:  None type:  <class 'str'>
referenced_tweet_id:  None type:  <class 'str'>
text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


<h3>Löschen von unnötigen Informationen</h3>
<p>Hier werden Informationen aus den Tweets Texten, die nicht weiter benötigt werden. Bspw. Links</p>

In [4]:
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == 
                      df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_manager_de.loc[each]["text"]))))

In [120]:
#just for test
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == 
                      df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '',(re.sub('[#|$]', '', df_manager_de.loc[each]["text"])))

In [121]:
df_manager_de.iloc[0]["text"]

'Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  '

In [122]:
len(df_manager_de)

14451

<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
    <li>kicker</li>
    <li>fpö</li>
    <li>fpoe</li>
</ul>

In [123]:
not_used_words_list = ["bvb", "fußball", "fussball", "dortmund", "fpö", "fpoe", "kicker"]
tweet_id_without_context_list = []
for each in range(len(df_manager_de)):
    for word in not_used_words_list:
        if word in df_manager_de.iloc[each]["text"].lower():
            if df_manager_de.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_manager_de.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [124]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager_de.loc[(df_manager_de["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager_de["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<p>Löschen der nicht kontextbezogenen Tweets und den Index neu setzen um später keine Probleme mit dem Index zu bekommen.</p>

In [125]:
for each in tweet_id_without_context_list:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_manager_de = df_manager_de.reset_index(drop=True)

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [126]:
counter = 0
retweet_list = []
for each in range(len(df_manager_de)):
    if "retweeted" in df_manager_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5278


In [127]:
df_manager_de = df_manager_de.drop(retweet_list)
df_manager_de = df_manager_de.reset_index(drop=True)

<h3>Löschen von doppelten Tweets</h3>
<p>Alle Tweets, die dopplet im Datensatz enthalten sind, anhand der tweet_id werden gelöscht. Somit werden auch die zuvor aufgespalteten Listen wieder gelöscht</p>

In [128]:
df_manager_de = df_manager_de.drop_duplicates(subset=["tweet_id"])
df_manager_de = df_manager_de.reset_index(drop=True)

In [129]:
len(df_manager_de)

8720

<h1>Überprüfung der Wörter in den Tweets</h1>
<p>Ich prüfe wie oft die einzelnen Wörter in den jeweiligen Tweets vorkommen um ggf. nicht passende Wörter zu ermitteln.</p>
<p>Ladeb von deutschen Stop Wörtern um diese zu filter, da diese keinen wirklichen Mehrwert bieten. <br> Anschließend wird das Hanover Tagger Modell geladen um die Lemmatisierung durchzuführen um Wörter auf die Grundform zu reduzieren.</p>

In [13]:
german_stop_words = stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

<p>Lemmatisierung um zu den jeweiligen Wörtern die Grundform zu erhalten. Dies wird mit den ganzen Tweets gemacht umd die Wörter im Kontext betrachten zu können (?). </p>

In [14]:
tweet_list = df_manager_de["text"].tolist()
lemma_list = []

for tweet in tweet_list:
    lemma = [lemma for (word, lemma, pos) in tagger.tag_sent(tweet.split())]
    lemma_list.append(' '.join(lemma))

<p>Erstellen einer Liste mit allen Wörtern aus jeden einzelnen Tweet.</p>

In [15]:
text_list = []
for each in lemma_list:
    for word in re.sub('\W+', ' ', each).lower().split():
        if word not in german_stop_words:
            text_list.append(word)

<p>Die Liste wird nun zu einer Pandas Serie um hier leichter Informationen zu erhalten. Im Anschluss wird die Serie zu einem Dataframe, bei dem die Spalten "Word" und "Count" heißen.</p>

In [16]:
text_series = pd.Series(text_list)
df_word_count = text_series.value_counts().to_frame().reset_index().rename(columns={"index": "word", 0: "count"})

<p>Anzeigen von allen Wörtern, die öfter als 100 Mal in dem Datensatz vorkommen.</p>

In [17]:
pd.set_option('display.max_rows', None)
df_word_count.loc[df_word_count["count"] >= 100]

word  count
0              wirecard   4506
1                 braun   2779
2                markus   2290
3              marsalek   1064
4                  chef   1006
5                    ex    870
6                   jan    869
7                  jahr    820
8              vorstand    512
9                 aktie    484
10                 geld    479
11               sollen    476
12               müssen    474
13                  gut    466
14                 groß    451
15                   ja    448
16                schon    440
17                  mal    437
18                 mehr    433
19             betrogen    432
20              deutsch    397
21                geben    390
22              belügen    386
23                 frei    376
24        partnerschaft    370
25                  ceo    366
26          unternehmen    340
27                  dax    336
28               kommen    334
29                 ganz    323
30                immer    322
31                gehen    315
32                 wien    303
33                 herr    291
34                  via    290
35              manager    281
36             betrüger    277
37                 mann    277
38               kaufen    276
39                    1    257
40                sehen    250
41               lassen    250
42                heute    245
43                  neu    242
44                    2    235
45              bericht    229
46                 euro    226
47                sagen    225
48                 seit    220
49                davon    218
50                 wohl    207
51                 kpmg    205
52              skandal    205
53                  wdi    203
54             betrügen    202
55                lügen    201
56                 fake    201
57               wissen    199
58               zurück    198
59               treten    192
60       finanzminister    191
61                 kurz    191
62                  wer    191
63              einfach    190
64                warum    181
65                bafin    177
66     börsengeschichte    175
67        bilanzskandal    169
68              glauben    168
69          weggeschaut    166
70             visionär    163
71           öffentlich    163
72                 früh    161
73                  nie    161
74               laufen    157
75                 kurs    156
76               heutig    156
77          markusbraun    156
78                   ft    155
79               stehen    154
80                sicht    154
81                 zeit    153
82                  amp    151
83               riesig    149
84                viele    149
85                 fall    148
86                bitte    148
87             auftritt    148
88               vieles    146
89              stellen    146
90               wirken    145
91            milliarde    145
92            vertrauen    144
93               nahezu    144
94              grotesk    143
95           vielleicht    143
96                  ton    143
97              erzählt    142
98              bleiben    141
99                  tag    141
100         börsenstory    140
101            ehemalig    140
102                bank    137
103               woche    136
104              finden    133
105       vorstandschef    133
106                   9    133
107             konzern    132
108            aktionär    129
109                  ab    129
110             million    128
111               frage    127
112  staatsanwaltschaft    127
113             anleger    126
114              denken    126
115                   s    124
116              gerade    124
117                erst    124
118         philippinen    124
119               wegen    124
120          festnehmen    123
121             münchen    122
122                   5    121
123           schreiben    121
124                 tun    121
125             letzter    118
126          eigentlich    118
127            versagen    118
128               le

<p>Überprüfung ob bei auffälligen Wörtern oder Kürzeln die Texte keinen Kontext zu Wirecard haben.</p>
<p>Auffällig Begriffe</p>
<lu>
    <li>FPÖ</li>
    <li>amp</li>
    <li>wdi</li>
</lu>

In [21]:
for each in df_manager_de.loc[df_manager_de.text.str.lower().str.contains("fpö")]["text"]:
    print(each, '\n')

In [19]:
for each in df_manager_de.loc[df_manager_de.text.str.lower().str.contains("amp")]["text"]:
    print(each, '\n')

   Im Wort Bunte steckt Braun in allen Schattierungen drin,auch das Frauke&amp;Markus Braun 

Ein weiterer Goldsponsor der nativeadscamp ist die ForwardAdGroup, vertreten durch Ihren Speaker Markus Braun.  

 📈 Nutze den Moment &amp; handel einfach und schnell am Kryptomarkt   

eines der sehr wenigen Interviews von Wirecard CEO Markus Braun - natürlich für Ö1 &amp; das Mittagsjournal 

Image-Ranking:  top, Bayer- &amp;  flop. Wirecard-Chef  hat das beste Image unter den Chefs der 30 größten Dax-Konzerne, Werner Baumann von  &amp; Dieter Zetsche von Daimler haben das schlechteste.   

  Hallo Hr. Bremer,welchen Zwist meinen Sie?Bisher ging d. Aggression v. d. FT England aus,scheibchenweise &amp; immer sensationell aufgebauscht wohlweislich d. Effekte auf d. Markt &amp; anscheinend mit teilweise suspekter Beteiligung v. Leerverkäufern.Würde mich interessieren.Wirecard 

     Nein. Es ist für mich lediglich ein Signal, dass noch nicht alles ausgestanden ist. Auch wenn R&amp;T positive Er

In [20]:
for each in df_manager_de.loc[df_manager_de.text.str.lower().str.contains("wdi")]["text"]:
    print(each, '\n')

▶ Wirecard-Aktie: Insiderdeal - Vorstand Markus Braun kauft erneut zu: Bad Marienberg ( - Bei der Aktie des Zahlungsabwicklers Wirecard AG (ISIN: DE0007472060, WKN: 747206, Ticker-Symbol: WDI, Nasdaq OTC-Symbol:…  AKTIENCHECKde 

     Ermittlern liegt eine Aussage vor, wonach ein Zeuge vorab wusste, wann negative Berichte über den Zahlungsdienstleister erscheinen würden.  $WDI   --&gt; FAZ article big news potentially implcating FT criminally?  I said it smells BAD. 

Wirecard WDI Telko: Wenn das alles so stimmt wie es gesagt wurde, scheint es tatsächlich eher ein "Non-Event" zu sein, wie es CEO Markus Braun bezeichnet. Er fokussiert sich lieber auf das operative Geschäft und erwartet unverändert ein starkes Geschäftsjahr 2019! 

  same hereZumal mir für das Drama in zig Akten dann doch der Glauben an die WDI GF fehlt. 

     Am 12.3. verkauft und am 13.3 wieder gekauft. Nach einem erneuten Artikel verkaufte ich mit der Absicht am darauffolgenden Tag wieder tiefer einzusteigen. Vom Pri

<p>Der Begriff der in der Stichprobe keinen Bezug zu Wirecard hatte war <b>"FPÖ"</b>. Dementsprechend ist der Begriff oben in der ursprünlgichen Verarbeitung mit aufgenommen.</p>

<h1>Check User</h1>
<p>Überprüfe die User Daten nach Namen, die nichts mit dem Thema Wirecard zu tun haben. Auch Nutzer, die ggf. nur durch Zufall in die Abfrage gekommen sind, z.B. durch den gleichen Namen wie einer der Manager, sollen aus dem Datensatz entnommen werden.<br>Auffällige Namen sind <b>Markus Braun</b> und <b>Jan Marsalek</b>, da es sich um den CEO und den verantwortlichen Manager handelt.</p>

<p>Laden der Daten</p>

In [14]:
df_user = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/user.csv", sep=";")

<p>Prüfen nach allen Nutzern mit dem Namen Markus</p>

In [19]:
df_user.loc[df_user.name.str.lower().str.contains("markus")].drop_duplicates()

user_id         username                       name  \
3              3365669884      MarkusB1980               Markus Braun   
53              170698930     Magix_Markus               Magic Markus   
58     827452635057881088     _MarkusBraun               Markus Braun   
91              536335486        Advobraun            RA Markus Braun   
253             745165314         mvdweyer        Markus van de Weyer   
951   1174753444189671424  MarkusW28522797                Markus Wolf   
1053           1201441525      MarkusFasse               Markus Fasse   
1298   780754694419296256    markuscbraun1            Markus C. Braun   
1347  1053554251367174145        MWeingran             MarkusWeingran   
1506            855232346      MarkusBlume               Markus Blume   
1580            348646470   MarkusMetschke            Markus Metschke   
1777   960496784278683648   MarkusTiemann1             Markus Tiemann   
1926  1160868405429383168   Markus70044344               Markus Naier   
2011           4143865486     MarkusToehne               Markus Töhne   
2624             56951088          patolix             Markus Trenker   
2720  1019851548350910464       mag_markus    Mag. Markus Steinbacher   
3426            497968900     MarkusEhrler              Markus Ehrler   
3445            572140860       rapidler74               Markus Riedl   
3734           2473666566     bachsblueten                Markus Bach   
3841           4548026189     markusdelitz            Markus Delitz ✨   
4573           1193169414    KorinekMarkus             Markus Korinek   
4823             14162263     markuswaeger               Markus Wäger   
4888            443638494   markuskrainer1             markus krainer   
4953             46415147           kubant                     markus   
5125   897858675855040512  MarkusMahler512              Markus Mahler   
5141             38137133   Dr_MarkusMeier               Markus Meier   
5197            197528170        ms_nieder          Markus Niedermann   
5397             37611246       rieksmeier          Markus Rieksmeier   
5932  1220822278440615936  MarkusA28136072        Markus ponnyhof.Sol   
5980            348086065         Leineems                     Markus   
7290            178680931      msulzbacher          Markus Sulzbacher   
7329   851496036472324098     Markus_Krall           Dr. Markus Krall   
7574           1011258769        abwerzger           Markus Abwerzger   
8162            176053221         KompaLaw               Markus Kompa   
8262            148084651     mwolschlager         Markus Wolschlager   
8358             49411452         mpindeus             Markus Pindeus   
8434  1043410202744246272  ElLatin85341655               Huber Markus   
8519  1031158517024079877       devotedly9         Markus Bruckmüller   
8542            221144872     unsigned_int        Markus Lauterbacher   
9001           2382279415      MarkusGroll               Markus Groll   
9133           2988134535   NeunerOfficial        Markus Neuner (SPÖ)   
9322             21320873  markus_reichert            Markus Reichert   
9567           1145319596      MarkusRosch               Markus Rosch   
9605            137070802  Markus_Wolf_HVA  Generaloberst Markus Wolf   

      follower  following  tweet_cound  list_count  verified  \
3           82        181          620          13     False   
53         163        192        24324          18     False   
58       31414         88         1162          81      True   
91          17         93           16           0     False   
253       1141       1297        35732         199     False   
951          0          4            2           0     False   
1053      1688       1065         2299          53     False   
1298        85        232           89           0     False   
1347       176         50          325           3     False   
1506     12911        284         1333         196      True   
1580         5         44   

<p>Prüfen auf auffäliige Einträge bezüglich Jan Marksalek</p>

In [20]:
df_user.loc[df_user.name.str.lower().str.contains("jan")].drop_duplicates()

user_id         username  \
181              152703288   dejanjovicevic   
471     960630599601082368        JHendrikP   
626              571011183       SeraelJann   
791              427754037        clauci2nd   
1089    861221059747610624     proudtobehay   
1120              48455041        Willmroth   
1456   1169762222790385665   Jannis73502821   
1488    926892715706593280     JanKaiserICH   
1915            1009685810  DanielaLongding   
1946   1225022850735771648          JMoldau   
2155    748259499245510656       sparfindig   
2158            2152612894           jc8136   
2631              19072286         janboehm   
2647    804934696555704320    Jane_Doe40806   
2734            3353363103   JanDHildebrand   
3033   1273891164224139264  JanMuel28853482   
3111            3170249476       jannikkel6   
3307   1269540544537145344        AktienUni   
3450             427754037        clauci2nd   
3791            2782752599          JanWa94   
3797              14359266           knalli   
4159             384818445         JanHemme   
4421   1244642921313435648         janvp000   
4734    838611262137372674    Jan2345678901   
4896   1262484333010604039         FreseJan   
4922              40251964        Ja_gir100   
5017              52198884  FineJanCannibal   
5553              18584829       jan_hauser   
5796            2771331905          janvesp   
6309    960463104537657344        DjirSarai   
6632            2857610675       JankoTietz   
6683   1281873818433269760     JanBehrens16   
6684             504918158  jan_buehlbecker   
6733   1025796393129177088  klausdietermai1   
6765            1262597888        J_Ehemann   
7369   1171000359638188033     januszcwikla   
7984   1067221755813654528      jannemanncs   
9025            1260188096     KarlssonKann   
9435             787856124            8B_EC   
9512            3357661984        janvoss81   
9760            2224564819           __ljrk   
10002             63764458       janejane24   
10096           1261872733    AndreasSJanke   
10294            972233251    Jan_Heiermann   

                                                    name  follower  following  \
181                                      Dejan Jovicevic      1785       1449   
471                                                  Jan        61        143   
626                                           SeraelJann         5         18   
791          claudia janecek - food & medicine #Yemen 🇾🇪      3504       4269   
1089                                              Django        48        803   
1120                                     Jan Diesteldorf      1995        957   
1456                                              Jannis         2         95   
1488                                          Jan Kaiser         3         22   
1915                                       daniel janson        10        100   
1946                                          Jan Moldau         0         10   
2155                           spar|fin|dig : Jan Tietje       619        919   
2158                                                 jan        89        167   
2631                                  Jan Böhmermann 😷🦠🤨   2502132       2042   
2647                                            Jane Doe      4615       4081   
2734                                      Jan Hildebrand      2547        883   
3033                                         Jan Mueller         0          4   
3111                             Jan Nikkel (geboostert)       323        297   
3307                          Jan Wolters | AktienUni.de       434        286   
3450         claudia janecek - food & medicine #Yemen 🇾🇪      3504       4269   
3791                                         Jan Wagener        11         57   
3797                                         Jan Philipp       246        510   
4159                                           Jan Hemme      1270        279   
4421                                    

<p>Auffällige User_IDs:</p>
<ul>Markus Braun
    <li>3365669884 - MarkusB1980</li>
    <li>827452635057881088 - _MarkusBraun( - CWO von Wirecard)</li>
    <li>536335486 - Advobraun</li>
    <li>780754694419296256 - markuscbraun1</li>
</ul>
<ul>Jan (Marsalek)
    <li>19072286 - janboehm</li>
</ul>

<p>Anzeigen der Tweets der "auffälligen" Nutzer</p>

In [28]:
#3365669884 - MarkusB1980
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 3365669884]["text"]

Series([], Name: text, dtype: object)

In [29]:
#827452635057881088 - _MarkusBraun( - CWO von Wirecard)
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 827452635057881088]["text"]

Series([], Name: text, dtype: object)

In [30]:
#536335486 - Advobraun
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 536335486]["text"]

Series([], Name: text, dtype: object)

In [31]:
#780754694419296256 - markuscbraun1
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 780754694419296256]["text"]

Series([], Name: text, dtype: object)

In [32]:
#19072286 - janboehm
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 19072286]["text"]

Series([], Name: text, dtype: object)

<p>Nur bei zwei der Nutzer ist nach der Vorverarbeitung noch ein Tweet in der Liste. Diese beiden Nutzer tragen nichts zum Thema Wirecard bei und twittern nicht diesbezüglich. <br>Ich habe Jan bei der Suche nach Jan lediglich den Namen Jan Böhmermann geprüft, da es sich hierbei um eine bekannte Persönlichkeit handelt, die zu diesem aber auch zu vielen anderen Themen twittert und ich dies deswegen überprüfen wollte.<br><br>Prüfen auf die Nutzer, die besonders oft im Datensatz der Manager vorkommen.<br>Nutzer, deren Inhalt keinen Mehrwert bieten: </p>
<lu>
    <li>3365669884 - MarkusB1980</li>
    <li>536335486 - Advobraun</li>
</lu>
<p>Dabei handelt es sich um die einzigen Nutzer, zu denen noch Tweets nach der Vorverarbeitung übrig sind</p>

<p>Das Dataframe mit der Anzahl der jeweiligen Autoren wird um die Informationen aus dem Datensatz der User ergänzt.<br>Dabei soll gerpüft werden, ob auffällige Nutzer mit eigenartigen Namen auftauchen, die es zu prüfen gilt. Dabei wird vor allem auf Nutzer geachtet, die einen klaren Bezug zu anderen Theman haben oder die ggf. auffälligkeiten zu dem Thema haben könnten.</p>

In [26]:
df_manager_de["author_id"].value_counts().to_frame().reset_index().join(
    df_user.set_index("user_id"), on="index").drop_duplicates(subset=["index"]).rename(columns={"index": "author_id", "author_id": "count"}).head(11)

author_id  count         username  \
0   1118219555967520769    863    AMCtootheMoon   
1   1159698800790953984    192     blvddemoulin   
2            3029833842    175   Steinfireflint   
3            2195833356     89          Ch11110   
4    704753138146877440     65  csFraudAnalysis   
5   1100318139718414336     65       Tradefair6   
6            2280206959     59      FreieWeltEu   
7             172082646     57      Bonnetmaker   
8   1095261591203966976     55     Kaan24263371   
9             583334012     52       blockhero_   
10             60673185     50   EhssanKhazaeli   

                                           name  follower  following  \
0                                   AMCtoooMoon       274        221   
1                                 A.Barendrecht       159        964   
2                    Dr. Fred... no MD/PD/XYZ-D       465        323   
3                                         Ch111        21        149   
4                                         csfa.      1035        561   
5                                     Tradefair        44         33   
6   🎲 Policon automatisches Nachrichtenportal 👽       689       2229   
7                                    Götz Kluge       268        591   
8                                         Aslan        93        133   
9                                 blockhero.eth       549       1480   
10                            Ehssan Khazaeli        362          1   

    tweet_cound  list_count  verified                 created_at  
0         22555           1     False  2019-04-16 18:28:12+00:00  
1          5477           0     False  2019-08-09 05:31:55+00:00  
2         23065          12     False  2015-02-19 12:35:58+00:00  
3          1422           0     False  2013-11-15 11:36:59+00:00  
4         15064          29     False  2016-03-01 19:40:16+00:00  
5          1766           1     False  2019-02-26 08:54:22+00:00  
6       1570308          66     False  2014-01-07 07:14:28+00:00  
7         15730           2     False  2010-07-28 21:40:20+00:00  
8          4525           1     False  2019-02-12 10:01:27+00:00  
9         15540           2     False  2012-05-18 01:19:29+00:00  
10         1599           4     False  2009-07-27 18:38:20+00:00

<p>Stichprobenartige Prüfen der einzelnen Top Nutzer mit mehr als 50 Tweets auf deren Inhalte.</p>

In [59]:
pd.set_option('display.max_colwidth', None)
for each in df_manager_de.loc[df_manager_de["author_id"] == 1118219555967520769].index:
    print(df_manager_de.iloc[each]["text"], "\n")

     Wir fragen? Wen haben Sie denn gefragt? Die FT? Armutszeugnis ist das sonst nichts. Und dafür zahle ich? Nicht mehr!!! 

 Respekt, dass Sie sich so beherrschen können - ich würde ausflippen bei der Berichterstattung. 💪 

     Wo bleiben Belege für Ihre wissentlich falschen Unterstellungen zu Wirecard? 

     Euch ist schon klar, dass dan euch nun einbindet und seine fakenews weiter verbreitet  

Hehres  für das Stimmvieh.              

 ist unser Top Pick für 2020 im DAX.Wir sehen das genau wie   Wirecard  

Wirecard Aktie: CEO Markus Braun verbreitet Zuversicht MobilePayment   

   Ich hoffe Sie morgen noch die Möglichkeit, über Ihren Broker bei L&amp;S handeln zu können...Denn der Montag dürfte für Aktionäre übel werden 🤭 

 Schade, der Kurs hat die sehr guten Zahlen nicht honoriert. WDI ist immer noch ein Schnäpchen im DAX. 

 Es stimmt mich sehr traurig , dass ich Euch Recht geben muss...Ich warte und vertraue Mr. Markus Braun 😊 Einmal muss man doch auf dem richtigen Weg sein

IndexError: single positional indexer is out-of-bounds

In [62]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 1159698800790953984].index:
    print(df_manager_de.iloc[each]["text"], "\n")

Wirecard-Vorstand Jan Marsalek ist höflich, zurückhaltend und unauffällig. Er ist zuständig für den Vertrieb - aber auch für das, was er "Feindaufklärung" nennt. Wie weit ist er dabei gegangen?Die Seite Drei von heute: $WDI 

Jan Marsalek sieht sich als 'Feindaufklärer'. Dubiose Kontakte tragen ihm offenbar Informationen zu. Treibt er es zu weit?Eine Geschichte über Spekulanten, einen Ex-Häftling, einen Ex-Agenten, einen möglichen Erpresser - und mittendrin Wirecard.  

Unter Wölfen: Wirecard -Vorstand Jan Marsalek betreibt nach eigenen Worten "Feindaufklärung" im Milieu Londoner Spekulanten. Recherche mit  , Klaus Ott,  ,    (via   

Wer wichtig werden wirdAuf diese CEOs sollten Sie 2020 achten  

     Wer WDI mit Steinhoff vergleicht, hat seine Reputation auf einen Schlag vernichtet. 

    Bzw Fake News 

     Ich bin voll bei  und  - besser als die anschuldigenden zurück anzuschuldigen wäre es, transparent zu den aufgeworfenen Fragen die Karten auf den Tisch zu legen 

 Guten Abend 

IndexError: single positional indexer is out-of-bounds

In [63]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 3029833842].index:
    print(df_manager_de.iloc[each]["text"], "\n")

  Wenn Wirecard sein Versprechen einlöst, diese Woche Aktien zurückzukaufen, dann kommt dem Unternehmen der Einbruch des Kurswertes vielleicht sogar recht gelegen und es müsste sich bei der FT bedanken ;-) 

   Warum dieser sarkastische Rahmen? 

    Man sieht wie tief du bist , Hab dich gemeldet hoffe Twitter sperrt dein Account, du kannst ja neu aufmachen 

  Unabhängig von Ihrer zusätzlichen Investition in das Unternehmen bin ich von Wirecart mehr als überzeugt. Wenn am 18.06. mit dem Bericht alle haltlosen Vorwürfe ausgeräumt. sind, hoffe, dass die Investoren sich auf das wesentliche konzentrieren. 

Kurssprung vor Geschäftsbericht: BaFin prüft Aktienkäufe des Wirecard-ChefsWirecard-Chef Markus Braun kauft kurz vor der beabsichtigten Veröffentlichung des Geschäftsberichts Aktien des eigenen Unternehmens im Wert von 2,5 Millionen Euro, der Kurs spri…  

     Wenn ich mache Presseartikel lese zur "Frühgeschichte" und dem Gesäusel, Markus Braun habe da eine Schrottfirma aus dem Münchn

IndexError: single positional indexer is out-of-bounds

In [64]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 2195833356].index:
    print(df_manager_de.iloc[each]["text"], "\n")

       Transparenzanspruch bei wirecard endet mit testierten Zahlen. War immer so, Markt glücklich, alles gut. "kleinreden und zum Kern der Sache schweigen. Die Masche hat immer funktioniert. Nun funktioniert sie nicht mehr" schreibt Wiwo. Abwarten.  

  Wirecard muss jetzt neue Bereiche erschließen das cloud-geschäft 

   Könnten Sie auch offenlegen, wie man versucht hat Sie für einen solchen Prozess zu gewinnen? 

  Ich hoffe nur, dass Wirecard wirklich sauber ist, weil für ein solches geniales Geschäftsmodell wäre das sehr schade. Ob man in einem Monat der beste ist oder in einem Jahr, spielt keine Rolle, wichtig ist Wirecard hat das potenziell, er muss nur richtig und würdig handeln. 

  Sag mal bist du von verdi bezahlt 😂 

   Ja, darum ging es mir. Danke. 

 Bitte zukünftig besser kommunizieren. Der EY Bericht war eine Katastrophe! So etwas geht in einem professionellen Unternehmen nicht. 

  Danke für die Info ;-) 

 Kaufen Sie Aktien nach? 

    ich denke aber dass die beiden K

IndexError: single positional indexer is out-of-bounds

In [65]:
for each in df_manager_de.loc[df_manager_de["author_id"] == 704753138146877440].index:
    print(df_manager_de.iloc[each]["text"], "\n")

8. Unser Mannschaftsarzt bleibt Dr. Markus Braun  

DLD13 - Future of mobile payment IV (René Schuster, Markus Braun, David Marcus)  staged  

    Dann kannst du noch nicht lange investiert sein. Alleine in den letzten 3 Jahren eine Performance von 197%. 

    Genau so nennt man es, verlogen...    und natürlich  hampelmann und dumm  ! Wie könnt ihr noch in den Spiegel schauen?! 

  Aber warum auch den Bericht veröffentlichen, wenn man auch bis zum 31. März warten kann? 

      ist ein Bulle eigentlich auch ein Ochse? 

            natürlich von wirecard um die Shortseller in eine Falle zu locken...kannst du alles im w:o forum nachlesen..😂hau dir aber vorher noch ne Buddel Rotwein rein.....😜 

            Werden wir ja sehen ?! Alles Taktik ! 

            Zum wievielten mal? Also wenn du investierst, dann tue dir den gefallen, hör auf damit. Es ist kein Glaubenskrieg. Wünsche jedem das Beste. Ich finde diese fanatische Einstellung pro oder kontra fehl im Aktienhandel. Morgen Absturz, f

IndexError: single positional indexer is out-of-bounds

<h3>Nutzer die keinen Mehrwert bieten werden gelöscht.</h3>

In [130]:
list_not_needed_author_ids = [3365669884, 536335486]
for each in list_not_needed_author_ids:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["author_id"] == each].index)
df_manager_de = df_manager_de.reset_index(drop=True)

<h1>Hinzufügen weiterer Twitter Datensätze</h1>

<h2>Datensatz mit Abfrage nach Wirecard</h2>

In [18]:
df_wirecard_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Wirecard_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wirecard_de = df_wirecard_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [19]:
for each in range(len(df_wirecard_de)):
    df_wirecard_de.loc[df_wirecard_de["text"] == 
                      df_wirecard_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_wirecard_de.loc[each]["text"]))))

In [20]:
df_wirecard_de.head()

conversation_id   author_id                created_at  \
0  704068304768524294    22018165 2016-02-28 22:18:59+00:00   
1  704044436297220096    25487236 2016-02-28 20:44:08+00:00   
2  704043629531275264    25487236 2016-02-28 20:40:56+00:00   
3  704043491379302400    25487236 2016-02-28 20:40:23+00:00   
4  704018583765000194  3000333467 2016-02-28 19:01:25+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  704068304768524294   de              9            0           0   
1  704044436297220096   de              9            0           0   
2  704043629531275264   de              3            0           0   
3  704043491379302400   de              4            0           0   
4  704018583765000194   de              0            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter for iPhone  retweeted  703316326803177473   
1            0    Twitter for iPad  retweeted  703316326803177473   
2            0    Twitter for iPad  retweeted  702851487383293952   
3            0    Twitter for iPad  retweeted  703317162430222336   
4            0  Twitter for iPhone       None                None   

                                                text  
0  RT  Die irrwitzige 67-Millionen-Euro-Wette des...  
1  RT  Die irrwitzige 67-Millionen-Euro-Wette des...  
2  RT  Sehr schön erklärt von HRD: Wirecard - Ang...  
3  RT  . kauft in 24 Monaten Aktien für das 38-Fa...  
4  3 lukrative Aktien im Quick-Check.True Gold/Wi...

<h3>Löschen von Retweets</h3>

In [21]:
counter = 0
retweet_list = []
for each in range(len(df_wirecard_de)):
    if "retweeted" in df_wirecard_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wirecard_de = df_wirecard_de.drop(retweet_list)
df_wirecard_de = df_wirecard_de.reset_index(drop=True)

54403


In [22]:
len(df_wirecard_de)

111531

<h3>Löschen von doppelten Einträgen</h3>

In [23]:
df_wirecard_de = df_wirecard_de.drop_duplicates(subset=["tweet_id"])
df_wirecard_de = df_wirecard_de.reset_index(drop=True)

In [24]:
len(df_wirecard_de)

111254

In [25]:
df_wirecard_de.head()

conversation_id   author_id                created_at  \
0  704018583765000194  3000333467 2016-02-28 19:01:25+00:00   
1  704016062682103808  3383023023 2016-02-28 18:51:23+00:00   
2  703952749650714624    20973609 2016-02-28 14:39:48+00:00   
3  703097227355430912  3296434535 2016-02-28 10:37:51+00:00   
4  703887151348457472   163466981 2016-02-28 10:19:09+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  704018583765000194   de              0            0           0   
1  704016062682103808   de              0            0           2   
2  703952749650714624   de              0            0           0   
3  703891857542127616   de              0            0           0   
4  703887151348457472   de              0            0           0   

   quote_count              source  tweet_type referenced_tweet_id  \
0            0  Twitter for iPhone        None                None   
1            0  Twitter Web Client        None                None   
2            0  Twitter for iPhone        None                None   
3            0  Twitter Web Client  replied_to  703101750123720704   
4            0              Aktien        None                None   

                                                text  
0  3 lukrative Aktien im Quick-Check.True Gold/Wi...  
1  Die Woche mit Fintech und Banken im Überblick ...  
2  Wirecard / Deutsche Telekom / DAX / Euro / Gol...  
3     weil wir letztens darüber geschrieben haben...  
4  Aktien, Banken, Makrodaten, S&amp;P 500 - Bärm...

<h2>Abfrage nach Chashtag Wirecard</h2>

In [26]:
df_wrcdf_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/WRCDF_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wrcdf_de = df_wrcdf_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [27]:
for each in range(len(df_wrcdf_de)):
    df_wrcdf_de.loc[df_wrcdf_de["text"] == 
                      df_wrcdf_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_wrcdf_de.loc[each]["text"]))))

In [28]:
len(df_wrcdf_de)

33

<h3>Löschen von Retweets</h3>

In [29]:
counter = 0
retweet_list = []
for each in range(len(df_wrcdf_de)):
    if "retweeted" in df_wrcdf_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wrcdf_de = df_wrcdf_de.drop(retweet_list)
df_wrcdf_de = df_wrcdf_de.reset_index(drop=True)

0


<h3>Löschen von doppelten Tweets</h3>

In [30]:
df_wrcdf_de = df_wrcdf_de.drop_duplicates(subset=["tweet_id"])
df_wrcdf_de = df_wrcdf_de.reset_index(drop=True)

In [31]:
len(df_wrcdf_de)

33

<h1>Zusammenführen der Datensätze</h1>

In [32]:
list_data_frames = [df_manager_de, df_wirecard_de, df_wrcdf_de]
df_de = pd.concat(list_data_frames, ignore_index=True)

In [33]:
print(len(df_manager_de))
print(len(df_wirecard_de))
print(len(df_wrcdf_de))
print(len(df_manager_de)+len(df_wirecard_de)+len(df_wrcdf_de))
print(len(df_de))

8725
111254
33
120012
120012


<h3>Löschen der doppelten Posts</h3>
<p>Dadurch, dass die Abfrage durch mehrere Querys erfolgt ist, befinden sich einige Tweets doppelt in dem Datensatz. Diese werden hier anhand der <b>"tweet_id"</b> gelöscht. </p>

In [34]:
df_de = df_de.drop_duplicates(subset=["tweet_id"])
df_de = df_de.reset_index(drop=True)

<p>Der neue Datensatz wird auf plausibilität geprüft.</p>

In [35]:
print(len(df_de))

114464


In [36]:
df_de.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  701784983829143553   554491983 2016-02-22 15:05:53+00:00   
3  700578193334411264    19811837 2016-02-19 07:10:32+00:00   
4  700221272840601601    19811837 2016-02-18 07:32:15+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  701784983829143553   de              0            0           0   
3  700578193334411264   de              2            0           0   
4  700221272840601601   de              0            0           1   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter Web Client       None                None   
1            0  Twitter Web Client       None                None   
2            0            Facebook       None                None   
3            0  Twitter Web Client       None                None   
4            0  Twitter Web Client       None                None   

                                                text  
0  Wirecard-Chef Markus Braun pumpt irrwitzige Su...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Morgen startet die neue Reihe R²-JazzCube! Vol...  
3  Wirecard: Was führt der Vorstand im Schilde? M...  
4  Wirecard-Vorstand kauft weiter massiv eigene A...

In [39]:
df_de.tail()

conversation_id            author_id                created_at  \
114463  1164960547693105153           4584354895 2019-08-23 18:00:13+00:00   
114464  1228994498610503680   704753138146877440 2020-02-17 17:53:29+00:00   
114465  1228017316375539712  1138485557502382082 2020-02-13 18:09:16+00:00   
114466  1275070899822059523           1001192245 2020-06-22 14:19:26+00:00   
114467  1271435871364042754            392388185 2020-06-12 13:45:22+00:00   

                   tweet_id lang  retweet_count  reply_count  like_count  \
114463  1164960547693105153   de              0            0           0   
114464  1229463896425205768   de              0            0           2   
114465  1228018315785838593   de              0            0           0   
114466  1275070899822059523   de              0            0           0   
114467  1271438445500682244   de              0            1           9   

        quote_count              source  tweet_type  referenced_tweet_id  \
114463            0         tickwatcher        None                 None   
114464            0     Twitter Web App  replied_to  1229459682403725314   
114465            1  Twitter for iPhone  replied_to  1228017316375539712   
114466            0      TalkMarketsApp        None                 None   
114467            0     Twitter Web App  replied_to  1271435871364042754   

                                                     text  
114463  Top Gainers (&gt;30)UVXY JNUG DRIP OPYGY URR E...  
114464                   1/ Die Anwälte der Klägerseit...  
114465  Ist nicht der WRCDF LV Anteil mit zusätzlichen...  
114466  Olaf Scholz - The German Finance Minister Is D...  
114467   12.06. WDI              29,09 Mio.  +1,16% (v...

In [43]:
df_de.iloc[2]

conversation_id                                       701784983829143553
author_id                                                      554491983
created_at                                     2016-02-22 15:05:53+00:00
tweet_id                                              701784983829143553
lang                                                                  de
retweet_count                                                          0
reply_count                                                            0
like_count                                                             0
quote_count                                                            0
source                                                          Facebook
tweet_type                                                          None
referenced_tweet_id                                                 None
text                   Morgen startet die neue Reihe R²-JazzCube! Vol...
Name: 2, dtype: object

<p><b>Fertig</b> mit der Vorverarbeitung der Daten in deutsch. Es ist nun ein Datensatz verfügbar mit allen Tweets für den deutschsprachigen Raum. Es sind keine doppelten Tweets enthalten, solange diese keine gleiche tweet_id enthalten. Auch Nutzer und tweets, die nichts zu dem eigentlichen Thema Wirecard beitragen sind nicht mehr im Datensatz enthalten. Vereinzelt können natürlich unnütze Tweets vorhanden sein, aber die Nutzer mit vielen Interaktionen sind überprüft und auch die Begriffe, die besonders oft vorkommen sind überprüft worden, ob diese etwas mit dem Thema zu tun haben.<br>Jetzt muss das gleich noch mit den Englischen tweets geschehen und dann müssen Datensätze zur Überprüfung ausgegeben werden.<br> Um den zufälligen Datensatz zu erstellen sollte ich noch einen zufallszahlengenerator nutzen, sodass ich möglichst zufällige Tweets zum testen erhalte. Hier scheint die Pandas Funktion <b>"sample"</b> gut geeignet zu sein. </p>

In [86]:
df_validation_sentiment = df_de.sample(n = 1500)

In [87]:
df_validation_sentiment.head()

conversation_id   author_id                created_at  \
64381   1265679986503155714    15765982 2020-05-27 16:23:18+00:00   
107551  1281257074501001219   322556713 2020-07-13 09:02:38+00:00   
87867   1273718320315936774   399025508 2020-06-18 20:44:46+00:00   
84562   1274337300760137728    69164225 2020-06-20 13:44:23+00:00   
46764   1125358812104282113  1289565265 2019-05-06 11:17:03+00:00   

                   tweet_id lang  retweet_count  reply_count  like_count  \
64381   1265679986503155714   de              0            0           0   
107551  1282601319958818817   de              0            0           0   
87867   1273718320315936774   de              2            1           3   
84562   1274337300760137728   de              0            0           2   
46764   1125358812104282113   de              0            0           0   

        quote_count              source  tweet_type  referenced_tweet_id  \
64381             0             infoyou        None                 None   
107551            0     Twitter Web App  replied_to  1281257074501001219   
87867             0           TweetDeck      quoted  1202606884781465600   
84562             0  Twitter for iPhone        None                 None   
46764             0             dlvr.it        None                 None   

                                                                                                                                                                                                                                          text  
64381                                                                                                                           Gute Nachrichten für Wirecard AG (WDI․DE) Aktionäre: Kursplus winkt (2020-05-27) Wirecard  Aktienkurs  infoyou  
107551                                                                                                                                                                              skandal  die beiden haben das Wirecard Prinzip aufgedekt    
87867                                                             Das war vor einem halben Jahr. Die BaFin muss sehr viele unangenehme Fragen beantworten. Bin gespannt, wer im Zuge des Wirecard-Skandals noch alles seinen Hut nehmen muss.   
84562                                                                                                      Heute Nacht über Nürnberg gewitter alter_schweede fritec spannung wirecard blitz kalchreuth franken Energiewende Nuernberg charger   
46764   ▶ Wirecard meist gehandelte Aktie an der Tradegate Exchange - Aktiennews: Frankfurt ( - Die Aktie der Wirecard AG (ISIN: DE0007472060, WKN: 747206, Ticker-Symbol: WDI, Nasdaq OTC-Symbol: WRCDF) war mit einem Umsatz…  AKTIENCHECKde

<h1>TEST</h1>

In [71]:
test_str = "Das ist ein Test. Ich Füge hier ein paar #Sonderzeichen $ein um zu sehen, wie ich diese §Löschen kann!"

In [72]:
print(test_str)

Das ist ein Test. Ich Füge hier ein paar #Sonderzeichen $ein um zu sehen, wie ich diese §Löschen kann!


In [75]:
re.sub('[$|§|#]', '', test_str)

'Das ist ein Test. Ich Füge hier ein paar Sonderzeichen ein um zu sehen, wie ich diese Löschen kann!'

In [76]:
count = 0
for each in range(len(df_de)):
    if "$" in df_de.iloc[each]["text"]:
        count += 1
print(count)

1208


In [85]:
df_de.sample(n = 100)

conversation_id            author_id                created_at  \
96787   1287740598074695680             14784765 2020-07-27 13:24:20+00:00   
56084   1206552703872196611            259733922 2019-12-16 12:32:35+00:00   
49938   1159150370926530561             41346564 2019-08-07 17:12:39+00:00   
72068   1276533199951323143              5560422 2020-06-26 15:10:06+00:00   
57257   1221729604836253698             44144586 2020-01-27 09:40:11+00:00   
54105   1197420676568768512            122035491 2019-11-21 07:45:10+00:00   
51689   1186180679526158336   743798562593640448 2019-10-21 20:34:28+00:00   
38900   1092332275822460928              2271711 2019-02-04 08:01:23+00:00   
97798   1286712990033678337           1114194576 2020-07-24 17:20:57+00:00   
103531  1284496853841707008             24868354 2020-07-18 14:34:49+00:00   
81622   1275070551040524292           2285678941 2020-06-22 14:18:03+00:00   
32105   1055480851759271936            389794039 2018-11-01 13:26:24+00:00   
8675    1278383962503688193             81550369 2020-07-01 17:44:22+00:00   
7607    1284523203784826882            160499861 2020-07-18 16:19:31+00:00   
15019    801717454351269888           3019835512 2016-11-24 09:21:50+00:00   
95584   1288431622921031680   904796734341120000 2020-07-29 11:10:11+00:00   
90085   1273546912529092609            281418458 2020-06-18 09:23:39+00:00   
45067   1113403294343471104            122035491 2019-04-03 11:30:06+00:00   
94980   1288491001422774272  1239515792011649024 2020-07-29 18:46:36+00:00   
83388   1274887337982509056            122035491 2020-06-22 02:10:02+00:00   
26633   1030126173974671360           2266388972 2018-08-16 16:16:33+00:00   
81816   1275051265869918211            915317144 2020-06-22 13:01:25+00:00   
13556    766533197643395072             48424772 2016-08-19 07:12:10+00:00   
59633   1227984207286296576           2410892498 2020-02-13 17:43:23+00:00   
26056   1034354826396790789             42484888 2018-08-28 08:19:42+00:00   
21133    933963611894054912            169251474 2017-11-24 07:40:51+00:00   
67378   1259543898512334848  1228707359557652480 2020-05-10 18:00:41+00:00   
88634   1273636380179075073             43872301 2020-06-18 15:19:10+00:00   
71193   1276814964993523714            523125803 2020-06-27 09:49:44+00:00   
89019   1273592939344809986           3029833842 2020-06-18 13:19:01+00:00   
10007    707161347629445120           4423529297 2016-03-08 11:09:38+00:00   
24690   1009318449074622464             19709133 2018-06-20 06:14:05+00:00   
84426   1274374312640856064   704753138146877440 2020-06-20 16:36:24+00:00   
3706    1276253940066258944             20592690 2020-06-25 20:40:25+00:00   
102876  1284822419430137858            239405776 2020-07-19 12:08:30+00:00   
36172   1097401355642257410             37382601 2019-02-18 07:44:06+00:00   
32875   1071091304417894409             52421660 2018-12-07 17:17:21+00:00   
79614   1275387537708781575   822072073468186624 2020-06-23 11:17:39+00:00   
46591   1126032799847321601            806696101 2019-05-08 07:55:14+00:00   
66529   1260874009513607168           1289565265 2020-05-14 10:06:04+00:00   
73674   1276243625635168265            227619477 2020-06-25 21:11:07+00:00   
104897  1284055459528421378   964856420549189633 2020-07-17 09:20:52+00:00   
56364   1204821978382839808            114508061 2019-12-11 17:55:18+00:00   
30376   1049249317276581888            122035491 2018-10-08 10:45:06+00:00   
62909   1254828054800666624           3029833842 2020-04-27 17:51:27+00:00   
61009   1237323523770720257            124205250 2020-03-10 10:24:51+00:00   
73677   1276181692001247232             60846161 2020-06-25 21:09:04+00:00   
47532   1139495377827172357             94581404 2019-06-14 11:30:43+00:00   
9883     707918792828067840             52421660 2016-03-10 13:19:27+00:00   
7151    1285902421647122433           2280206959 2020-07-22 11:40:02+00:00   
75511   1276112680248541186       

In [15]:
test_str_strInStr = "Das ist der Test ob Superkicker korrekt erkannt wird."

In [16]:
if "kicker" in test_str_strInStr:
    print("Superkicker")

Superkicker


In [131]:
df_markus = df_manager_de.loc[df_manager_de.text.str.lower().str.contains("markus braun|markusbraun")].reset_index(drop=True)

In [132]:
df_jan = df_manager_de.loc[df_manager_de.text.str.lower().str.contains("jan marsalek|janmarsalek")].reset_index(drop=True)

In [60]:
df_markus.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  700578193334411264    19811837 2016-02-19 07:10:32+00:00   
3  700221272840601601    19811837 2016-02-18 07:32:15+00:00   
4  697324084221227008   183289696 2016-02-10 07:39:51+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  700578193334411264   de              2            0           0   
3  700221272840601601   de              0            0           1   
4  697324084221227008   de              0            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter Web Client       None                None   
1            0  Twitter Web Client       None                None   
2            0  Twitter Web Client       None                None   
3            0  Twitter Web Client       None                None   
4            0             dlvr.it       None                None   

                                                text  
0  Wirecard-Chef Markus Braun pumpt irrwitzige Su...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Wirecard: Was führt der Vorstand im Schilde? M...  
3  Wirecard-Vorstand kauft weiter massiv eigene A...  
4  📈 Wirecard: Serie von Aktienkäufen des Vorstan...

In [61]:
df_jan.head()

conversation_id            author_id                created_at  \
0  1095335360928702465             20592690 2019-02-12 14:54:35+00:00   
1  1094967519910203392             20592690 2019-02-11 14:32:54+00:00   
2  1094899104537206784           1289565265 2019-02-11 10:01:03+00:00   
3  1093604642964074496  1092881249889370113 2019-02-07 20:17:19+00:00   
4  1093540135029927938   771328590373871617 2019-02-07 16:00:59+00:00   

              tweet_id lang  retweet_count  reply_count  like_count  \
0  1095335360928702465   de              0            0           0   
1  1094967519910203392   de              0            0           0   
2  1094899104537206784   de              0            0           0   
3  1093604642964074496   de              0            0           0   
4  1093540135029927938   de              1            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0             Echobox       None                None   
1            0             Echobox       None                None   
2            0             dlvr.it       None                None   
3            0  Twitter Web Client       None                None   
4            1  Twitter Web Client       None                None   

                                                text  
0  Alexander von Knoop und Jan Marsalek haben die...  
1  Alexander von Knoop und Jan Marsalek haben die...  
2  ▶ Wirecard-Vorstände kaufen Aktien: Alexander ...  
3  Wirecard: Heute haben die beiden Vorstände Ale...  
4  Wirecard AG: Nach der neuesten Short-Attacke b...

In [133]:
print(len(df_markus))
print(len(df_jan))
print(len(df_markus)+len(df_jan))
print(len(df_manager_de))

7148
899
8047
8716


In [134]:
list_markus_jan = []
for each in range(len(df_markus)):
    list_markus_jan.append(df_markus.iloc[each]["tweet_id"])
for each in range(len(df_jan)):
    list_markus_jan.append(df_jan.iloc[each]["tweet_id"])

In [135]:
print(len(df_markus))
print(len(df_jan))
print(len(list_markus_jan))

7148
899
8047


In [136]:
list_markus_jan[0:5]

[703311366262800384,
 702405572012974080,
 700578193334411264,
 700221272840601601,
 697324084221227008]

In [137]:
df_manager_de.loc[df_manager_de["tweet_id"] == 700578193334411264]

conversation_id  author_id                created_at  \
3  700578193334411264   19811837 2016-02-19 07:10:32+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
3  700578193334411264   de              2            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
3            0  Twitter Web Client       None                None   

                                                text  
3  Wirecard: Was führt der Vorstand im Schilde? M...

In [145]:
list_index = []
for each in list_markus_jan:
    list_index.append(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
    
df_man_with_mark_jan = df_manager_de.drop(list_index)
    #print(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_man_with_mark_jan = df_man_with_mark_jan.reset_index(drop=True)

In [146]:
len(df_man_with_mark_jan)

713

In [140]:
for each in range(0,100):
    print(df_man_with_mark_jan.iloc[each]["text"])

Morgen startet die neue Reihe R²-JazzCube! Vol. 1 - Das wird toll und wir noch toller, wenn DU/IHR auch komm(s)t!... 
Deutschland: Überrollt vom selbstfahrenden Auto 
@"Deutschland: Überrollt vom selbstfahrenden Auto"  auf @LinkedIn
Deutschland: Überrollt vom selbstfahrenden Auto 
Lesenswerter Überblick zu @wirecard bei @faznet (Paywall via @digitec 
Kennst du Wirecard-Chef Markus (Braun)? Für ihn regnet es derzeit Lobeshymnen, von denen Elon (Musk) nur träumen kann. Manager Fintech Österreicher 
Konzernchef im Interview: Herr Braun, garantieren Sie, dass Wirecard sauber ist? 
Konzernchef im Interview: Herr Braun, garantieren Sie, dass Wirecard sauber ist? 
Der Wirecard Chef gibt sein erstes großes Interview, natürlich in @DerSPIEGEL 
Konzernchef im Interview: Herr Braun, garantieren Sie, dass Wirecard sauber ist? 
Konzernchef im Interview: Herr Braun, garantieren Sie, dass Wirecard sauber ist?  
Konzernchef im Interview: Herr Braun, garantieren Sie, dass Wirecard sauber ist?  
Wirecar

In [158]:
df_with_wirecard = df_man_with_mark_jan.loc[df_man_with_mark_jan.text.str.lower().str.contains("wirecard")]

In [165]:
len(df_with_wirecard)

519

In [167]:
list_with_wirecard = []
for each in range(len(df_with_wirecard)):
    list_with_wirecard.append(df_with_wirecard.iloc[each]["tweet_id"])

In [168]:
len(list_with_wirecard)

519

In [171]:
list_index = []
for each in list_with_wirecard:
    list_index.append(df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == each].index[0])
    #print(df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == each].index[0])
df_man_with_mark_jan = df_man_with_mark_jan.drop(list_index)
df_man_with_mark_jan = df_man_with_mark_jan.reset_index(drop=True)

In [172]:
len(df_man_with_mark_jan)

194

In [176]:
for each in range(len(df_man_with_mark_jan)):
    print(df_man_with_mark_jan.iloc[each]["tweet_id"], ": ", df_man_with_mark_jan.iloc[each]["text"])

701784983829143553 :  Morgen startet die neue Reihe R²-JazzCube! Vol. 1 - Das wird toll und wir noch toller, wenn DU/IHR auch komm(s)t!... 
909831998537158657 :  Deutschland: Überrollt vom selbstfahrenden Auto 
909733713302441984 :  @"Deutschland: Überrollt vom selbstfahrenden Auto"  auf @LinkedIn
909730731932368897 :  Deutschland: Überrollt vom selbstfahrenden Auto 
1037447116665364480 :  Der ganz andere Konzernchef  münchen munich
1047546831142567936 :  Kannste dir nicht ausdenken. 
1092469534198583297 :  Ein Milliardär in der Defensive 
1108640951218696193 :  Jetzt nennt die "Financial Times" Namen. 
1121374023013761024 :  @UMilatz @FelixHoltermann @handelsblatt Und Braun hat darauf schon reagiert in einem Interview:
1115628103487954944 :  Köpfe-Quiz für Experten: Diese Leute sollten Sie kennen - tun Sie's auch? 
1115596390133051393 :  Köpfe-Quiz für Experten: Diese Leute sollten Sie kennen - tun Sie's auch? 
1112624842493358080 :  Spirituosenexperte für hochprozentigen Genuss 
1131